In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import scipy.signal as signal
import torch

In [2]:
#print(torch.cuda.is_available())
#print(torch.cuda.get_device_name(0))

In [3]:
#  TODO: Add functionality to handle joining multiple csvs together from different file locations into usable df

In [ ]:
path = "/data1/dnicho26/EMG_DATASET/data/data/10/steps/1740933999.5057008.csv"

df = pd.read_csv(path, header=None)
df.columns = df.iloc[0]
df = df.drop(0, axis=0).reset_index(drop=True)


FileNotFoundError: [Errno 2] No such file or directory: '/data1/dnicho26/EMG_DATASET/data/data/DS1/10/steps/1740933999.5057008.csv'

In [27]:
# Drop unnecessary columns
columns_to_drop = [col for col in df.columns if isinstance(col, str) and ('IMP' in col.upper() or 'TIME' in col.upper()) and col != 'time 0']
df = df.drop(columns=columns_to_drop)
df = df.drop(df.columns[0], axis=1)
df = df.apply(pd.to_numeric, errors='coerce')

In [28]:
df = df[200:390000]

In [29]:
df

,time 0,sensor 0 EMG 1 (1259.259 Hz),sensor 0 ACC X (148.148 Hz),sensor 0 ACC Y (148.148 Hz),sensor 0 ACC Z (148.148 Hz),sensor 0 GYRO X (148.148 Hz),sensor 0 GYRO Y (148.148 Hz),sensor 0 GYRO Z (148.148 Hz),sensor 1 EMG 1 (1259.259 Hz),sensor 1 ACC X (148.148 Hz),...,sensor 3 GYRO X (148.148 Hz),sensor 3 GYRO Y (148.148 Hz),sensor 3 GYRO Z (148.148 Hz),sensor 4 EMG 1 (1259.259 Hz),sensor 4 ACC X (148.148 Hz),sensor 4 ACC Y (148.148 Hz),sensor 4 ACC Z (148.148 Hz),sensor 4 GYRO X (148.148 Hz),sensor 4 GYRO Y (148.148 Hz),sensor 4 GYRO Z (148.148 Hz)
200,1738798000.493261575698853,0.000000000000000,-0.086547851562500,-0.980468750000000,-0.263000488281250,13.251908302307127,-0.450381666421890,0.847328245639801,0.000000000000000,-0.092224121093750,...,-15.603053092956545,3.213740348815918,-1.267175555229187,0.000000000000000,0.020812988281250,-1.000244140625000,0.057617187500000,-12.564885139465332,13.389312744140623,2.748091697692871
201,1738798000.500011682510376,0.000000000000000,-0.083007812500000,-0.980407714843750,-0.265808105468750,13.229007720947266,-0.526717543601990,1.015267133712769,0.000000000000000,-0.089233398437500,...,-15.694656372070312,3.175572633743286,-1.442748069763184,0.000000000000000,0.021118164062500,-1.001892089843750,0.059082031250000,-12.412213325500488,13.740458488464355,2.549618244171143
202,1738798000.506761789321899,0.000000000000000,-0.083435058593750,-0.981079101562500,-0.267456054687500,13.343511581420898,-0.358778625726700,1.244274854660034,0.000000000000000,-0.092712402343750,...,-15.748091697692873,2.137404680252075,-1.503816843032837,0.000000000000000,0.019165039062500,-1.004211425781250,0.054931640625000,-12.091603279113770,13.977099418640137,2.534351110458374
203,1738798000.513511657714844,0.000000000000000,-0.079650878906250,-0.982666015625000,-0.267944335937500,13.137404441833496,-0.282442748546600,1.320610642433167,0.000000000000000,-0.099609375000000,...,-15.694656372070312,0.755725204944611,-1.137404561042786,0.000000000000000,0.020080566406250,-1.003540039062500,0.051513671875000,-11.877862930297852,14.412213325500488,2.557251930236816
204,1738798000.520261764526367,0.000000000000000,-0.077697753906250,-0.984436035156250,-0.267517089843750,12.770992279052734,-0.099236644804478,1.297709941864014,0.000000000000000,-0.101379394531250,...,-15.839694976806641,-0.267175585031509,-1.106870174407959,0.000000000000000,0.021545410156250,-0.999084472656250,0.048522949218750,-11.633587837219238,14.854961395263672,2.496183156967163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389995,NaN,-0.041794460975051,NaN,NaN,NaN,NaN,NaN,NaN,-0.011245899137865,NaN,...,NaN,NaN,NaN,-0.028702220187686,NaN,NaN,NaN,NaN,NaN,NaN
389996,NaN,-0.042298008697642,NaN,NaN,NaN,NaN,NaN,NaN,-0.010910200656138,NaN,...,NaN,NaN,NaN,-0.036087586785687,NaN,NaN,NaN,NaN,NaN,NaN
389997,NaN,-0.044480048828870,NaN,NaN,NaN,NaN,NaN,NaN,-0.011749446860456,NaN,...,NaN,NaN,NaN,-0.039276722362096,NaN,NaN,NaN,NaN,NaN,NaN
389998,NaN,-0.047837033646143,NaN,NaN,NaN,NaN,NaN,NaN,-0.011413748378729,NaN,...,NaN,NaN,NaN,-0.040451667048142,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
emg_columns = ['sensor 0 EMG 1 (1259.259 Hz)',
               'sensor 1 EMG 1 (1259.259 Hz)',
               'sensor 2 EMG 1 (1259.259 Hz)',
               'sensor 3 EMG 1 (1259.259 Hz)',
               'sensor 4 EMG 1 (1259.259 Hz)',
               'sensor 5 EMG 1 (1259.259 Hz)']

In [31]:
def resample_data(df, target_fs):
    """
    Resample data to the target sampling frequency.
    
    Parameters:
    - df: The DataFrame containing the time and signal data (e.g., 'time 0', EMG, GYRO, ACC columns).
    - target_fs: The target sampling frequency (Hz).
    
    Returns:
    - df_resampled: A new DataFrame with all columns resampled to the target frequency.
    """
    time_column = df['time 0'].dropna()
    if len(time_column) < 2:
        raise ValueError("Time column 'time 0' has fewer than 2 valid entries after removing NaN.")
    
    #  Calculate total duration and original sampling frequency
    total_duration = time_column.iloc[-1] - time_column.iloc[0]  # Total time span
    if np.isnan(total_duration):
        raise ValueError("Total duration is NaN. Check 'time 0' for invalid values.")
    
    original_length = len(time_column)  #  Number of valid samples in time column
    original_fs = original_length / total_duration  #  Average original sampling frequency (Hz)
    
    #  Calculate target length based on target_fs
    target_length = int(total_duration * target_fs)  #  Number of samples at target frequency
    if target_length <= 0:
        raise ValueError(f"Target length {target_length} is invalid. Check total_duration and target_fs.")
    
    new_time = np.linspace(time_column.iloc[0], time_column.iloc[-1], target_length)  # New time array
    
    df_resampled = pd.DataFrame({'time 0': new_time})
    
    emg_columns = [col for col in df.columns if 'EMG' in col]
    gyro_accel_columns = [col for col in df.columns if 'GYRO' in col or 'ACC' in col]
    
    #  Resample all signal columns to the target length
    for col in emg_columns + gyro_accel_columns:
        original_signal = df[col].dropna()
        if len(original_signal) < 2:
            print(f"Warning: Column '{col}' has fewer than 2 valid entries. Filling with zeros.")
            df_resampled[col] = np.zeros(target_length)
            continue
        
        #  Interpolate to match time column length if needed
        if len(original_signal) != original_length:
            original_time = np.linspace(time_column.iloc[0], time_column.iloc[-1], len(original_signal))
            interpolated_signal = np.interp(time_column, original_time, original_signal)
            resampled_signal = signal.resample(interpolated_signal, target_length)
        else:
            resampled_signal = signal.resample(original_signal, target_length)
        df_resampled[col] = resampled_signal
    
    return df_resampled

In [32]:
#  Implement butterworth filters of different orders, cutoff frequencies, sampling frequencies, and filter_band type to columns of a dataframe.

def butterworth_filter(data, order, cutoff, fs, filter_band='low'):
    nyquist = 0.5 * fs
    normalized_cutoff = np.array(cutoff) / nyquist
    
    #  Create Butterworth filter coefficients
    if filter_band == 'low':
        b, a = signal.butter(order, normalized_cutoff[0], btype='low')
    elif filter_band == 'high':
        b, a = signal.butter(order, normalized_cutoff[0], btype='high')
    elif filter_band == 'bandpass':
        b, a = signal.butter(order, normalized_cutoff, btype='bandpass')
    elif filter_band == 'bandstop':
        b, a = signal.butter(order, normalized_cutoff, btype='bandstop')
    else:
        raise ValueError(f"unsupported filter: {filter_band}")
    
    #  Apply filter
    filtered_data = signal.filtfilt(b, a, data)
    return filtered_data

def apply_filter(df, columns, order=4, cutoff=[1.0], fs=1000, filter_band='low'):
    '''
    Apply a Butterworth filter of specified order and cutoff to the given columns of the DataFrame.

    Parameters:
    - df: The dataframe containing signal data.
    - columns: List of columns to apply the filter to (e.g., EMG columns).
    - order: The order of the filter.
    - cutoff: The cutoff frequency or frequencies.
    - fs: Sampling frequency of the signal.
    - filter_band: Type of filter ('low', 'high', 'bandpass', 'bandstop').
    
    Returns:
    - df: The DataFrame with filtered columns.
    '''
    for col in columns:
        df[col] = butterworth_filter(df[col].values, order, cutoff, fs, filter_band)
    
    return df

In [33]:
def apply_notch(df, columns, notch_freq=60, fs=1000, Q=30):
    '''
    Apply a notch filter to remove a specific frequency (e.g., powerline noise at 60Hz)
    to the specified columns in the DataFrame.

    Parameters:
    - df: The dataframe containing signal data.
    - columns: List of columns to apply the notch filter to.
    - notch_freq: Frequency to remove with the notch filter (e.g., 60Hz for powerline noise).
    - fs: Sampling frequency of the signal.
    - Q: Quality factor for the notch filter.
    
    Returns:
    - df: The DataFrame with filtered columns.
    '''
    nyquist = 0.5 * fs
    normalized_frequency = notch_freq / nyquist

    b, a = signal.iirnotch(normalized_frequency, Q)

    for col in columns:
        df[col] = signal.filtfilt(b, a, df[col].values)
    
    return df

In [69]:
from scipy import signal

def resample_pose(df, fs):
    """
    Resample the YOLO keypoints DataFrame to 10 Hz.
    
    Parameters:
    - df: DataFrame with 'time 0' column and keypoint columns
    
    Returns:
    - df_resampled: DataFrame resampled to 10 Hz
    """
    time_column = df['time 0'].dropna()
    if len(time_column) < 2:
        raise ValueError("Time column 'time 0' has fewer than 2 valid entries after removing NaN.")
    
    total_duration = time_column.iloc[-1] - time_column.iloc[0]
    if np.isnan(total_duration) or total_duration <= 0:
        raise ValueError("Invalid total duration.")
    
    original_length = len(time_column)
    target_fs = fs  # Hz
    target_length = int(total_duration * target_fs)
    if target_length < 2:
        raise ValueError("Target length too short. Possibly not enough time range in data.")
    
    new_time = np.linspace(time_column.iloc[0], time_column.iloc[-1], target_length)
    df_resampled = pd.DataFrame({'time 0': new_time})
    
    keypoint_columns = [col for col in df.columns if col.startswith('kp')]

    for col in keypoint_columns:
        original_signal = df[col].dropna()

        # Handle columns that don't match original length
        if len(original_signal) != original_length:
            original_time = np.linspace(time_column.iloc[0], time_column.iloc[-1], len(original_signal))
            interpolated_signal = np.interp(time_column, original_time, original_signal)
            resampled_signal = signal.resample(interpolated_signal, target_length)
        else:
            resampled_signal = signal.resample(original_signal, target_length)

        df_resampled[col] = resampled_signal

    return df_resampled

# Pre-processing

In [70]:
#  Drop table if any column missing
if len(df.columns) < 43:
    print("yikes! having missing data is quite embarassing")
else:
    print("lets go")

#  Scale data with minimax
columns_non_time = df.columns[1:]
scaler_minmax = MinMaxScaler()
df[columns_non_time] = scaler_minmax.fit_transform(df[columns_non_time])

lets go


In [71]:
#  Resample data

df = resample_data(df, target_fs=100)

#  Apply butterworth filter
#df = apply_filter(df, emg_columns)

#  Apply notch filter
#f = apply_notch(df, emg_columns)

In [72]:
pd.set_option('display.float_format', lambda x: '%.15f' % x)

In [73]:
df['time 0']

0      1738796884.923694610595703
1      1738796884.933696985244751
2      1738796884.943699359893799
3      1738796884.953701972961426
4      1738796884.963704347610474
                  ...            
5491   1738796939.847003936767578
5492   1738796939.857006311416626
5493   1738796939.867008924484253
5494   1738796939.877011299133301
5495   1738796939.887013673782349
Name: time 0, Length: 5496, dtype: float64

# Pose Data Extraction

In [1]:
import cv2
import numpy as np
import pandas as pd
import time
import os
import io
import contextlib
from ultralytics import YOLO
import random
model = YOLO('yolov8n-pose.pt')

In [2]:
video_path = "/home/eric/Desktop/cv/steps/camera_2"
action_path = "/home/eric/Desktop/cv/steps/"
fps = 10

In [3]:
# Define keypoint indices (COCO format)
NOSE = 0
LEFT_EYE = 1
RIGHT_EYE = 2
LEFT_EAR = 3
RIGHT_EAR = 4
LEFT_SHOULDER = 5
RIGHT_SHOULDER = 6
LEFT_ELBOW = 7
RIGHT_ELBOW = 8
LEFT_WRIST = 9
RIGHT_WRIST = 10
LEFT_HIP = 11
RIGHT_HIP = 12
LEFT_KNEE = 13
RIGHT_KNEE = 14
LEFT_ANKLE = 15
RIGHT_ANKLE = 16

In [4]:
skeleton_connections = [
    (5, 7),  # left shoulder to left elbow
    (7, 9),  # left elbow to left wrist
    (6, 8),  # right shoulder to right elbow
    (8, 10), # right elbow to right wrist
    (5, 6),  # left shoulder to right shoulder
    (5, 11), # left shoulder to left hip
    (6, 12), # right shoulder to right hip
    (11, 13),# left hip to left knee
    (13, 15),# left knee to left ankle
    (12, 14),# right hip to right knee
    (14, 16),# right knee to right ankle
    (11, 12) # left hip to right hip
]


## Stream video with pose keypoints and knee flexion angles

In [5]:

def draw_keypoints(image, keypoints, confidence_threshold=0.3):
    
    num_keypoints = len(keypoints) // 3
    for i in range(num_keypoints):
        x = int(keypoints[i * 3])
        y = int(keypoints[i * 3 + 1])
        conf = keypoints[i * 3 + 2]
        if conf > confidence_threshold:
            cv2.circle(image, (x, y), 4, (0, 255, 0), -1)


def draw_skeleton(image, keypoints, skeleton_connections, confidence_threshold=0.3):
    for connection in skeleton_connections:
        idx1, idx2 = connection
        x1, y1, c1 = keypoints[idx1 * 3], keypoints[idx1 * 3 + 1], keypoints[idx1 * 3 + 2]
        x2, y2, c2 = keypoints[idx2 * 3], keypoints[idx2 * 3 + 1], keypoints[idx2 * 3 + 2]

        if c1 > confidence_threshold and c2 > confidence_threshold:
            pt1 = (int(x1), int(y1))
            pt2 = (int(x2), int(y2))
            cv2.line(image, pt1, pt2, (255, 0, 0), 2)


def calculate_angle(a, b, c):

    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    cb = c - b

    dot_product = np.dot(ab, cb)
    norm_ab = np.linalg.norm(ab)
    norm_cb = np.linalg.norm(cb)

    if norm_ab == 0 or norm_cb == 0:
        return 0.0

    cosine_angle = dot_product / (norm_ab * norm_cb)
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)

    angle_rad = np.arccos(cosine_angle)
    return np.degrees(angle_rad)

In [6]:
def stream_npy_video(directory, fps=10):
    files = sorted([f for f in os.listdir(directory) if f.endswith('.npy')])
    if not files:
        print("No .npy files found.")
        return

    cv2.namedWindow('YOLOv8 Pose Estimation', cv2.WINDOW_NORMAL)
    frame_interval = 1.0 / fps

    for file in files:
        start_time = time.time()
        frame = np.load(os.path.join(directory, file)).astype(np.uint8)

        if frame.shape[-1] != 3:
            print(f"Invalid frame format in {file}. Skipping.")
            continue

        results = model(frame, verbose=False)
        result = results[0]

        if result.keypoints is not None:
            keypoints = result.keypoints.data.cpu().numpy().flatten()

            draw_keypoints(frame, keypoints)
            draw_skeleton(frame, keypoints, skeleton_connections)

            def get_point(index):
                return (keypoints[index * 3], keypoints[index * 3 + 1])

            points = {
                'left_hip': get_point(LEFT_HIP),
                'right_hip': get_point(RIGHT_HIP),
                'left_knee': get_point(LEFT_KNEE),
                'right_knee': get_point(RIGHT_KNEE),
                'left_ankle': get_point(LEFT_ANKLE),
                'right_ankle': get_point(RIGHT_ANKLE),
                'left_shoulder': get_point(LEFT_SHOULDER),
                'right_shoulder': get_point(RIGHT_SHOULDER),
                'left_elbow': get_point(LEFT_ELBOW),
                'right_elbow': get_point(RIGHT_ELBOW),
                'left_wrist': get_point(LEFT_WRIST),
                'right_wrist': get_point(RIGHT_WRIST),
                'nose': get_point(NOSE)
            }

            def midpoint(p1, p2):
                return ((p1[0] + p2[0]) / 2, (p1[1] + p2[1]) / 2)

            angles = {
                'Left Knee': calculate_angle(points['left_ankle'], points['left_knee'], points['left_hip']),
                'Right Knee': calculate_angle(points['right_ankle'], points['right_knee'], points['right_hip']),
                'Left Hip': calculate_angle(points['left_knee'], points['left_hip'], points['left_shoulder']),
                'Right Hip': calculate_angle(points['right_knee'], points['right_hip'], points['right_shoulder']),
                'Left Elbow': calculate_angle(points['left_wrist'], points['left_elbow'], points['left_shoulder']),
                'Right Elbow': calculate_angle(points['right_wrist'], points['right_elbow'], points['right_shoulder']),
                'Pelvic Tilt': calculate_angle(points['left_hip'],
                                               midpoint(points['left_hip'], points['right_hip']),
                                               points['right_hip']),
                'Trunk Lean': calculate_angle(midpoint(points['left_hip'], points['right_hip']),
                                              midpoint(points['left_shoulder'], points['right_shoulder']),
                                              points['nose']),
            }

            angles['Knee Symmetry'] = abs(angles['Left Knee'] - angles['Right Knee'])
            angles['Hip Symmetry'] = abs(angles['Left Hip'] - angles['Right Hip'])
            angles['Elbow Symmetry'] = abs(angles['Left Elbow'] - angles['Right Elbow'])

            for i, (label, value) in enumerate(angles.items()):
                y = 40 + i * 30
                cv2.putText(frame, f"{label}: {value:.1f}°", (30, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        cv2.imshow('YOLOv8 Pose Estimation', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        elapsed = time.time() - start_time
        sleep_time = frame_interval - elapsed
        if sleep_time > 0:
            time.sleep(sleep_time)

    cv2.destroyAllWindows()

In [7]:
stream_npy_video(video_path)

## Extract averaged kinematic features from multiple video folders at each timestep

In [8]:
def extract_timestamp(filename):
    try:
        base = filename.split('_')[1]
        seconds = base.split('.')[0]
        milliseconds = base.split('.')[1]
        return float(f"{seconds}.{milliseconds}")
    except (IndexError, ValueError):
        return None

def calculate_angle(a, b, c):
    ab = np.array([a[0] - b[0], a[1] - b[1]])
    bc = np.array([c[0] - b[0], c[1] - b[1]])
    dot_product = np.dot(ab, bc)
    magnitude_ab = np.linalg.norm(ab)
    magnitude_bc = np.linalg.norm(bc)
    if magnitude_ab == 0 or magnitude_bc == 0:
        return 0.0
    angle_rad = np.arccos(np.clip(dot_product / (magnitude_ab * magnitude_bc), -1.0, 1.0))
    return np.degrees(angle_rad)

def midpoint(p1, p2):
    return ((p1[0] + p2[0]) / 2, (p1[1] + p2[1]) / 2)

def process_frame_split(frame):
    with contextlib.redirect_stdout(io.StringIO()):
        results = model(frame, verbose=False)
    for result in results:
        keypoints = result.keypoints.data.cpu().numpy().flatten()

        def get_point(index):
            return (keypoints[index * 3], keypoints[index * 3 + 1])

        points = {name: get_point(idx) for name, idx in {
            'left_hip': LEFT_HIP, 'right_hip': RIGHT_HIP,
            'left_knee': LEFT_KNEE, 'right_knee': RIGHT_KNEE,
            'left_ankle': LEFT_ANKLE, 'right_ankle': RIGHT_ANKLE,
            'left_shoulder': LEFT_SHOULDER, 'right_shoulder': RIGHT_SHOULDER,
            'left_elbow': LEFT_ELBOW, 'right_elbow': RIGHT_ELBOW,
            'left_wrist': LEFT_WRIST, 'right_wrist': RIGHT_WRIST,
            'nose': NOSE
        }.items()}

        # Angles
        left_knee_angle = calculate_angle(points['left_ankle'], points['left_knee'], points['left_hip'])
        right_knee_angle = calculate_angle(points['right_ankle'], points['right_knee'], points['right_hip'])

        left_hip_angle = calculate_angle(points['left_knee'], points['left_hip'], points['left_shoulder'])
        right_hip_angle = calculate_angle(points['right_knee'], points['right_hip'], points['right_shoulder'])

        left_elbow_angle = calculate_angle(points['left_wrist'], points['left_elbow'], points['left_shoulder'])
        right_elbow_angle = calculate_angle(points['right_wrist'], points['right_elbow'], points['right_shoulder'])

        pelvic_mid = midpoint(points['left_hip'], points['right_hip'])
        pelvic_tilt_angle = calculate_angle(points['left_hip'], pelvic_mid, points['right_hip'])

        hip_center = pelvic_mid
        shoulder_center = midpoint(points['left_shoulder'], points['right_shoulder'])
        trunk_lean_angle = calculate_angle(hip_center, shoulder_center, points['nose'])

        return {
            'left_knee': left_knee_angle,
            'right_knee': right_knee_angle,
            'left_hip': left_hip_angle,
            'right_hip': right_hip_angle,
            'left_elbow': left_elbow_angle,
            'right_elbow': right_elbow_angle,
            'pelvic_tilt': pelvic_tilt_angle,
            'trunk_lean': trunk_lean_angle,
            'knee_symmetry': abs(left_knee_angle - right_knee_angle),
            'hip_symmetry': abs(left_hip_angle - right_hip_angle),
            'elbow_symmetry': abs(left_elbow_angle - right_elbow_angle)
        }
    return None

def process_video_folder_split(folder_path):
    files = sorted([f for f in os.listdir(folder_path) if f.endswith('.npy')],
                   key=lambda f: extract_timestamp(f) if extract_timestamp(f) is not None else float('inf'))
    data = []

    for file in files:
        timestamp = extract_timestamp(file)
        if timestamp is None:
            continue
        frame = np.load(os.path.join(folder_path, file)).astype(np.uint8)
        if frame.shape[-1] != 3:
            continue
        angles = process_frame_split(frame)
        if angles:
            angles['time'] = timestamp
            data.append(angles)

    return pd.DataFrame(data)

def resample_split(df, target_fs=10):
    if len(df) < 2:
        return df

    start_time = df['time'].iloc[0]
    end_time = df['time'].iloc[-1]
    duration = end_time - start_time
    target_len = int(duration * target_fs)

    if target_len < 2:
        return df

    new_time = np.linspace(start_time, end_time, target_len)
    resampled = {'time': new_time}

    for column in df.columns:
        if column != 'time':
            resampled[column] = np.interp(new_time, df['time'], df[column])

    return pd.DataFrame(resampled)

def process_all_folders_split(parent_folder, target_fs=10):
    all_dfs = []
    for folder in sorted(os.listdir(parent_folder)):
        folder_path = os.path.join(parent_folder, folder)
        if os.path.isdir(folder_path):
            df = process_video_folder_split(folder_path)
            if not df.empty:
                df_resampled = resample_split(df, target_fs=target_fs)
                all_dfs.append(df_resampled)

    if not all_dfs:
        return pd.DataFrame()

    merged_df = pd.concat(all_dfs).groupby('time', as_index=False).mean()
    return merged_df


In [13]:
angles_df = process_all_folders_split(action_path)

## Dateframe we get returned:

In [14]:
angles_df

,time,left_knee,right_knee,left_hip,right_hip,left_elbow,right_elbow,pelvic_tilt,trunk_lean,knee_symmetry,hip_symmetry,elbow_symmetry
0,1.738797e+09,177.242385,176.365951,171.611144,99.097012,178.285251,0.000000,179.999960,161.002134,0.876434,72.514132,178.285251
1,1.738797e+09,12.185554,0.000000,176.741535,0.000000,177.506600,0.000000,180.000000,115.081315,12.185554,176.741535,177.506600
2,1.738797e+09,171.157284,172.355071,171.965861,156.055383,176.935541,0.000000,179.999971,113.610393,1.525789,15.910478,176.935541
3,1.738797e+09,108.191380,100.498888,171.689840,104.929723,171.856322,93.154156,179.999969,114.730214,7.692491,77.071655,78.702166
4,1.738797e+09,168.506301,171.807761,173.064770,165.244079,178.974689,118.855260,179.999992,118.856450,3.301459,7.820692,60.119429
...,...,...,...,...,...,...,...,...,...,...,...,...
1094,1.738797e+09,164.761253,156.869338,171.855259,173.284296,166.500941,161.261840,179.999996,142.137363,7.891915,1.481947,5.239101
1095,1.738797e+09,161.889157,163.510550,168.215731,172.342051,167.480533,165.541025,180.000000,141.927953,1.621393,4.126319,5.650225
1096,1.738797e+09,153.940343,148.928337,174.271010,174.205628,168.734615,160.769935,179.999996,141.122677,5.012007,0.065383,7.964679
1097,1.738797e+09,154.742769,154.337374,170.667639,171.118981,167.870319,170.857895,180.000000,144.177900,0.699266,1.688341,8.603936


In [20]:
angles_df = angles_df.replace(0, np.nan)
angles_df = angles_df.interpolate()
angles_df = angles_df.fillna(method='bfill').fillna(method='ffill')

/tmp/ipykernel_55711/2332684146.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  angles_df = angles_df.fillna(method='bfill').fillna(method='ffill')


In [21]:
angles_df

,time,left_knee,right_knee,left_hip,right_hip,left_elbow,right_elbow,pelvic_tilt,trunk_lean,knee_symmetry,hip_symmetry,elbow_symmetry
0,1.738797e+09,177.242385,176.365951,171.611144,99.097012,178.285251,93.154156,179.999960,161.002134,0.876434,72.514132,178.285251
1,1.738797e+09,12.185554,174.360511,176.741535,127.576198,177.506600,93.154156,180.000000,115.081315,12.185554,176.741535,177.506600
2,1.738797e+09,171.157284,172.355071,171.965861,156.055383,176.935541,93.154156,179.999971,113.610393,1.525789,15.910478,176.935541
3,1.738797e+09,108.191380,100.498888,171.689840,104.929723,171.856322,93.154156,179.999969,114.730214,7.692491,77.071655,78.702166
4,1.738797e+09,168.506301,171.807761,173.064770,165.244079,178.974689,118.855260,179.999992,118.856450,3.301459,7.820692,60.119429
...,...,...,...,...,...,...,...,...,...,...,...,...
1094,1.738797e+09,164.761253,156.869338,171.855259,173.284296,166.500941,161.261840,179.999996,142.137363,7.891915,1.481947,5.239101
1095,1.738797e+09,161.889157,163.510550,168.215731,172.342051,167.480533,165.541025,180.000000,141.927953,1.621393,4.126319,5.650225
1096,1.738797e+09,153.940343,148.928337,174.271010,174.205628,168.734615,160.769935,179.999996,141.122677,5.012007,0.065383,7.964679
1097,1.738797e+09,154.742769,154.337374,170.667639,171.118981,167.870319,170.857895,180.000000,144.177900,0.699266,1.688341,8.603936


# Pose extraction end

# Modeling

In [17]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [18]:
def split_dataframe(df):
    time_column = df.columns[0]
    
    sensor_columns = df.columns[1:]

    left_keywords = ['sensor 0', 'sensor 1', 'sensor 2']

    left_sensors = [col for col in sensor_columns if any(keyword in col for keyword in left_keywords)]
    right_sensors = [col for col in sensor_columns if col not in left_sensors]

    left_emg = [col for col in left_sensors if 'EMG' in col]
    right_emg = [col for col in right_sensors if 'EMG' in col]

    left_inputs = [col for col in left_sensors if col not in left_emg]
    right_inputs = [col for col in right_sensors if col not in right_emg]

    return df[left_inputs], df[left_emg], df[right_inputs], df[right_emg]

In [19]:
def create_sliding_windows(features, targets, window_size, target_size, step_size):
    X, y = [], []

    for i in range(0, len(features) - window_size - target_size + 1, step_size):
        X.append(features.iloc[i:i + window_size].values)
        y.append(targets.iloc[i + window_size:i + window_size + target_size].values)

    return np.array(X), np.array(y)

In [20]:

# Start with df with shape (number_of_samples, number_of_columns)

left_sensors, left_emg, right_sensors, right_emg = split_dataframe(df)

#  Small window, step, target for real-time prediction??

window_size = 10
step_size = 1
target_size = 5

#  Sensors and EMG used to predict other leg EMG

left_combined = pd.concat([left_sensors, left_emg], axis=1)
right_combined = pd.concat([right_sensors, right_emg], axis=1)
X_left, y_left = create_sliding_windows(left_combined, right_emg, window_size, target_size, step_size)
X_right, y_right = create_sliding_windows(right_combined, left_emg, window_size, target_size, step_size)
#  Shape = (batch_size, sequence_length, input_size)


#  TODO: determine the real-time delay from prediction to signals transmission sent so that we know how to optimize these parameters. 
#  For example, this delay will affect choice of target size because we might need multiple predictions in advance. 

input_size = 21  #  Number of features per time step (left sensors + left EMG)
output_size = 3   #  Number of output features
target_size = 5   #  Number of time steps in target

hidden_size = 64  #  Number of hidden units
num_layers = 2    #  Number of LSTM layers


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, target_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.target_size = target_size
    
    def forward(self, x):
        out, _ = self.lstm(x)  # Out has shape (batch_size, seq_length, hidden_size)
        out = out[:, -self.target_size:, :]  #  Get the output for the last `target_size` time steps
        out = self.fc(out)  #  Apply the output layer for each time step
        return out

#  Train/test split 
X_left_train, X_left_temp, y_right_train, y_right_temp = train_test_split(X_left, y_right, test_size=0.2, random_state=42)
X_left_val, X_left_test, y_right_val, y_right_test = train_test_split(X_left_temp, y_right_temp, test_size=0.5, random_state=42)

#  Convert to PyTorch tensors
X_left_train_tensor = torch.tensor(X_left_train, dtype=torch.float32)
y_right_train_tensor = torch.tensor(y_right_train, dtype=torch.float32)
X_left_val_tensor = torch.tensor(X_left_val, dtype=torch.float32)
y_right_val_tensor = torch.tensor(y_right_val, dtype=torch.float32)
X_left_test_tensor = torch.tensor(X_left_test, dtype=torch.float32)
y_right_test_tensor = torch.tensor(y_right_test, dtype=torch.float32)

model = LSTMModel(input_size, hidden_size, num_layers, output_size, target_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

patience = 25  #  Number of epochs to wait for improvement
best_loss = float('inf')  #  Initialize the best loss to infinity
patience_counter = 0  #  Counter to track how many epochs we've waited without improvement
best_model_wts = None  #  To store the best model weights

#  Training loop with validation and testing (left leg predict right leg)
num_epochs = 100
for epoch in range(num_epochs):
    model.train() # Training mode
    
    #  Forward pass (training)
    outputs_train = model(X_left_train_tensor)  # Get model predictions
    loss_train = criterion(outputs_train, y_right_train_tensor)  #  Compute loss on training data
    
    #  Backward pass and optimization
    optimizer.zero_grad()  #  Clear previous gradients
    loss_train.backward()  #  Backpropagate gradients
    optimizer.step()  #  Update model parameters
    
    #  Validation
    model.eval()  #  Eval mode
    with torch.no_grad():  #  Disable gradient calculation for validation
        outputs_val = model(X_left_val_tensor)  #  Get model predictions on validation data
        loss_val = criterion(outputs_val, y_right_val_tensor)  #  Compute loss on validation data
    
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {loss_train.item():.6f}, "
          f"Validation Loss: {loss_val.item():.6f}")

    #  Early stopping logic
    if loss_val < best_loss:
        best_loss = loss_val
        patience_counter = 0  #  Reset patience counter if validation loss improved
        best_model_wts = model.state_dict()  #  Save the model weights
    else:
        patience_counter += 1  #  Increment patience counter if no improvement
    
    #  If the patience counter exceeds the threshold, stop training
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break

#  After training is complete, test the model
model.eval()  # Eval mode
with torch.no_grad():  #  Disable gradient calculation for testing
    outputs_test = model(X_left_test_tensor)  #  Get model predictions on test data
    loss_test = criterion(outputs_test, y_right_test_tensor)  #  Compute loss on test data

print(f"Test Loss: {loss_test.item():.6f}")


KeyboardInterrupt: 

In [ ]:
#  Save the model weights
torch.save(model.state_dict(), 'lstm_weights_1.pth')